# Mohammed Murtuza Bhaiji - 1001666586

In [35]:
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup
import sqlite3

def getTickersFromCNN():
    
    #Extract the most active tickers from money.cnn.com
    try:
        url='https://money.cnn.com/data/hotstocks/'
        hotstocks_handle = requests.get(url)    # requests for the given URL
        
        # web scraping and parsing the response using beautifulsoup to find the desired elements.
        soup = BeautifulSoup(hotstocks_handle.content, 'html.parser')  
        active_stocks = soup.find('table') # fids the table element in the web page and returns an iterating object.
    
        stocksList = [] # storing all data as a list of dictionaries.
        for elem in active_stocks: # iterating every rows of table and finding the ticker and company names.
            stocks_dictionary = {}
            symbol = elem.find('a')
            company_name = elem.find('span')
            
            # handling none-type and object mismatch exceptions.
            if None in (symbol, company_name):
                continue
            if -1 in (symbol, company_name):
                continue

            symbol = symbol.text. strip()     # striping data from their respective tags.
            company_name = company_name.text.strip()
            
            # storing data as list of dictionaries.
            stocks_dictionary["Ticker"] = symbol
            stocks_dictionary["Company Name"] = company_name
            stocksList.append(stocks_dictionary)


        return stocksList
    except requests.exceptions.RequestException as e: # Handle all the request Exceptions 
        print("Request Error.! Please check the money.cnn.com website.")
        
def getDataFromYahoo(active_stocks_list):
    
    try:
        tickerurl = 'https://finance.yahoo.com/quote/{myticker}?p={myticker}&.tsrc=fin-srch-v1'

        for company in active_stocks_list:
            ticker = company["Ticker"]
            mystock_handle=requests.get(tickerurl.format(myticker = ticker))  # requests for the given URL

            # web scraping and parsing the response using beautifulsoup to find the desired elements.
            soup = BeautifulSoup(mystock_handle.content, 'html.parser')
            stocks_details = soup.find('table') # fids the table element in the web page and returns an iterating object.
            
            # keeping required attributes in a list
            attributes = ['OPEN-value', 'PREV_CLOSE-value', 'TD_VOLUME-value']
            
            # Iterating the soup object to find our desired attributes.
            for att in attributes:
                for cols in soup.findAll('td', attrs={'data-test': att}): # finds the attribute based on value we pass
                        for span in cols.findAll('span', recursive=False):
                            if(att == "TD_VOLUME-value"):
                                volume = span.text.strip()   # striping data from their respective tags.
                                # removing ',' from the number as volume needs to be stored as Integer.
                                volume = volume.translate({ord(','): None})
                                company[att] = volume
                            else:
                                company[att] = span.text.strip()   # striping data from their respective tags.
        return active_stocks_list
    except requests.exceptions.RequestException as e: # Handle all the request  Exceptions 
        print("Request error.! Please check the finance.yahoo.com website.")
    
def savingDataToText(active_stocks_list):
    # storing data in a .txt file
    try:
        file = open("stocks.txt","w")
    #     columns = ",".join(active_stocks_list[0].keys()) + "\n"
    #     file.write(columns)
        
        # iterating our main list of dictioaries 
        for company in active_stocks_list:
        # writing values of each dictionaries as single line in the text file.
            record = ",".join(company.values()) + "\n"
            file.write(record)
        file.close()   # closing the file.
        print("Data saved to stocks.txt")
    except:
        print("file error while saving data to .txt")
    
def savingDataToSqlite(active_stocks_list):
    # storing in Sqlite
    try:
        # connects to a db file and creates if it doesn't exist.
        conn = sqlite3.connect('StocksDatabase.db')
        cur = conn.cursor()
        cur.execute('DROP TABLE IF EXISTS StocksTable')  # droping the table if it exists
        # creating a new StocksTable in db.
        cur.execute('CREATE TABLE StocksTable (Ticker TEXT, OpenPrice REAL, PrevClose REAL, Volume INTEGER)')
        
        # parsing data from stocks.txt and inserting it line by line.
        file = open("stocks.txt", "r")
        for line in file:
            line = line.strip("\n")
            columns = line.split(",")
            ticker = columns[0]
            openprice = columns[2]
            closeprice = columns[3]
            volume = columns[4]
            
            # inserting values in DB.
            cur.execute('INSERT INTO StocksTable (Ticker, OpenPrice, PrevClose, Volume) VALUES (?,?,?,?)',
                       (ticker, openprice, closeprice, volume))
            conn.commit()

        cur.close()
        conn.close()
        print("Data saved to StockDatabase.db")
    
    except Exception as err:  # handling exceptions.
        print('Query Failed:', str(err))
    finally:
        conn.close()
        
        
def printData(active_stocks_list, printFlag):
    if(printFlag == "tickers"):
        print (f"{'Ticker'} \t{'Company Name'}")
    else:
        print (f"{'Ticker'} \t{'Company Name'} \t{'OpenPrice'} \t{'PrevClose'} \t{'Volume'}")
               
    for d in active_stocks_list:
        vals = list(d.values()) 
        for i in range(len(vals)):
           print (f"{vals[i]}", end="\t")
        print("", end="\n")
        

if __name__ == "__main__":
    print("scraping CNN data for most active stocks...", end="\n")
    print(end="\n")
    print("MOST ACTIVE STOCKS")
    active_stocks_list = getTickersFromCNN()
    printData(active_stocks_list, "tickers")
    print(end="\n")
    
    print("getting stock details from yahoo finance...")
    print(end="\n")
    print("MOST ACTIVE STOCK DETAILS")
    active_stocks_list = getDataFromYahoo(active_stocks_list)
    printData(active_stocks_list, "stockdetails")
    print(end="\n")
    
    print("saving data to stocks.txt")
    savingDataToText(active_stocks_list)
    print(end="\n")
    
    print("saving data in sqlite...")
    savingDataToSqlite(active_stocks_list)
    print(end="\n")
    
    print("All done. Goodbye!!")
    
    
    

scraping CNN data for most active stocks...

MOST ACTIVE STOCKS
Ticker 	Company Name
GE	General Electric Co	
F	Ford Motor Co	
BAC	Bank of America Corp	
WFC	Wells Fargo & Co	
DAL	Delta Air Lines Inc	
NCLH	Norwegian Cruise Line Holdings Ltd	
ORCL	Oracle Corp	
CCL	Carnival Corp	
MRO	Marathon Oil Corp	
BMY	Bristol-Myers Squibb Co	

getting stock details from yahoo finance...

MOST ACTIVE STOCK DETAILS
Ticker 	Company Name 	OpenPrice 	PrevClose 	Volume
GE	General Electric Co	5.61	5.70	127686754	
F	Ford Motor Co	4.8000	4.8900	76071585	
BAC	Bank of America Corp	20.29	21.71	52801954	
WFC	Wells Fargo & Co	23.69	24.06	51561673	
DAL	Delta Air Lines Inc	18.97	19.38	43349693	
NCLH	Norwegian Cruise Line Holdings Ltd	10.52	10.76	44252256	
ORCL	Oracle Corp	52.70	52.91	19348561	
CCL	Carnival Corp	11.67	12.27	36018920	
MRO	Marathon Oil Corp	5.62	5.64	34275970	
BMY	Bristol-Myers Squibb Co	64.20	63.77	22085825	

saving data to stocks.txt
Data saved to stocks.txt

saving data in sqlite...
Data saved to Sto